In [1]:
%matplotlib inline
import sys
import os
import io
import math
import pathlib
import logging
import numpy
import pandas
import psycopg2
from psycopg2 import extras

In [2]:
_logger = logging.getLogger( __name__ )
_logger.propagate = False
_logout = logging.StreamHandler( sys.stderr )
_logger.addHandler( _logout )
_logout.setFormatter( logging.Formatter( f'[%(asctime)s - %(levelname)s] - %(message)s' ) )
_logger.setLevel( logging.INFO )

In [3]:
with open( pathlib.Path(os.getenv("HOME")) / 'secrets' / 'decatdb_desi_desi' ) as ifp:
    dbuser,dbpassword = ifp.readline().strip().split()
dbargs = { 'host': 'decatdb.lbl.gov',
           'port': 5432,
           'dbname': 'desidb',
           'user': dbuser,
           'password': dbpassword,
           'cursor_factory': psycopg2.extras.RealDictCursor }

In [4]:
dir = pathlib.Path( os.getenv('HOME') ) / "desi/mosthosts"
newsnlist = pandas.read_csv( dir / 'data_master_clean.csv' )
dfs = []
for i in range(0,10):
    dfs.append( pandas.read_csv( dir / f'master_April2023_host_dil10_{i}.csv', low_memory=False ) )

# the sn_name_sp column in additional_host_April2023.csv has some problems; patch them
addl = pandas.read_csv( dir / f'additional_host_April2023.csv', low_memory=False )
w = addl.program == 'Pantheon'
addl.loc[w, 'sn_name_sp'] = addl[w].program + addl[w].sn_name
w = addl.program == 'DECAT'
addl.loc[w, 'sn_name_sp'] = addl[w].sn_name
w = addl.program.str[0:7] == 'legaObj'
addl.loc[w, 'sn_name_sp'] = addl[w].sn_name
dfs.append( addl )

newhostlist = pandas.concat( dfs, ignore_index=True )

FileNotFoundError: [Errno 2] No such file or directory: '/global/homes/r/raknop/desi/mosthosts/data_master_clean.csv'

In [10]:
newhostlist.loc[ numpy.where( newhostlist['ra_dr9'].isnull() ) ]

,origin,sn_name,sn_name_sp,program,ls_id_dr9,dec_dr9,ra_dr9,elat_dr9,elon_dr9,glat_dr9,...,brickname_sga,ra_sga,dec_sga,d26_sga,d26_ref_sga,pa_sga,ba_sga,ra_moment_sga,dec_moment_sga,sma_moment_sga
40037,local_universe,09azf,PTF09azf,PTF-iPTF,NaN,NaN,NaN,NaN,NaN,NaN,...,2472p407,247.437029,40.811627,5.14426,SB26,141.045,0.620638,247.437321,40.811298,104.629


In [11]:
# There is a single object that has a null ra_dr9 and dec_dr9
newhostlist['ra'] = newhostlist['ra_dr9']
newhostlist['dec'] = newhostlist['dec_dr9']
wnull = newhostlist['ra'].isnull()
newhostlist.loc[wnull, 'ra'] = newhostlist[wnull]['ra_sga']
newhostlist.loc[wnull, 'dec'] = newhostlist[wnull]['dec_sga']

In [12]:
db = psycopg2.connect( **dbargs )
cursor = db.cursor()
_logger.info( "Creating temp table..." )
cursor.execute( "CREATE TEMP TABLE newradec(id bigint,spname text,ra double precision,dec double precision)" )
strio = io.StringIO()
_logger.info( "Building copy input..." )
for i, row in newhostlist.iterrows():
    strio.write( f'{i}\t{row.sn_name_sp}\t{row.ra}\t{row.dec}\n' )
strio.seek(0)
_logger.info( "Copying to temp table..." )
cursor.copy_from( strio, 'newradec' )
_logger.info( "Running join..." )
cursor.execute( "SELECT n.id,n.spname,n.ra,n.dec,"
                "  t.targetid,t.release,t.ra AS desira,t.dec AS desidec,t.desi_target,t.bgs_target,t.mws_target "
                "FROM newradec n "
                "INNER JOIN general.maintargets t ON q3c_join(n.ra,n.dec,t.ra,t.dec,0.2/3600.) "
                "ORDER BY spname,targetid " )
rows = cursor.fetchall()
_logger.info( "Join done, pandafying..." )
comparison = pandas.DataFrame( rows )
cursor.close
db.rollback()
_logger.info( 'Done' )
db.close()

[2023-05-15 08:31:11,379 - INFO] - Creating temp table...
[2023-05-15 08:31:11,383 - INFO] - Building copy input...
[2023-05-15 08:31:15,311 - INFO] - Copying to temp table...
[2023-05-15 08:31:15,376 - INFO] - Running join...
[2023-05-15 08:31:16,723 - INFO] - Join done, pandafying...
[2023-05-15 08:31:16,862 - INFO] - Done


In [13]:
print( f'{len(comparison)} of {len(newhostlist)} hosts are on the DESI main targets list' )
print( f'{(comparison["bgs_target"] != 0).sum()} have non-0 bgs_target' )
print( f'{(comparison["mws_target"] != 0).sum()} have non-0 mws_target' )
print( f'{(comparison["desi_target"] != 0).sum()} have non-0 desi_target' )

30199 of 60574 hosts are on the DESI main targets list
28120 have non-0 bgs_target
137 have non-0 mws_target
30199 have non-0 desi_target


In [14]:
numpy.sqrt( ( (comparison['dec'] - comparison['desidec']) ** 2 ).mean() )*3600

0.0064827800918661775

In [15]:
len(newhostlist)

60574

In [16]:
comparison

,id,spname,ra,dec,targetid,release,desira,desidec,desi_target,bgs_target,mws_target
0,483,2004ht,16.203306,-0.894884,39627760838122472,9010,16.203306,-0.894884,1152921504606846976,131074,0
1,484,2004ht,16.203527,-0.894906,39627760838122481,9010,16.203527,-0.894906,5764607523034234880,131074,0
2,482,2004ht,16.204548,-0.896522,39627760838122513,9010,16.204548,-0.896522,1152921504606846976,65537,0
3,11782,2004hv,357.103267,1.063071,39627814877531796,9010,357.103267,1.063071,65537,0,0
4,11781,2004hv,357.109800,1.064573,39627814877532048,9010,357.109800,1.064573,5764607523034234880,131074,0
...,...,...,...,...,...,...,...,...,...,...,...
30194,51453,ZTF21acoqcje,120.655757,5.326055,39627913489811337,9010,120.655757,5.326055,65537,0,0
30195,16881,ZTF21acoscof,37.170498,9.610367,39628014014696795,9010,37.170498,9.610367,1179842,0,0
30196,28470,ZTF21acpemhp,154.676060,23.274060,39628334379830705,9010,154.676060,23.274060,1152921504606846976,131074,0
30197,45705,ZTF21acqbdvg,110.803013,50.835527,39633258928932184,9011,110.803013,50.835527,4738,0,0


In [17]:
db.close()